# KubeFlow Pipeline Using TFX OSS Components

In this notebook, we will demo: 

* Defining a KubeFlow pipeline with Python DSL
* Submiting it to Pipelines System
* Customize a step in the pipeline

We will use a pipeline that includes some TFX OSS components such as [TFDV](https://github.com/tensorflow/data-validation), [TFT](https://github.com/tensorflow/transform), [TFMA](https://github.com/tensorflow/model-analysis).

## Setup

In [1]:
# our minio credentials
S3_ENDPOINT = 'minio.app.cluster1.demo01cloud.dev.superhub.io'
S3_ACCESS_KEY = 'minio'
S3_SECRET_KEY = 'minio1234'
S3_BUCKET = 'default'
#S3_ENDPOINT = 's3.us-east-1.amazonaws.com'

# Set your output and project. !!!Must Do before you can proceed!!!
EXPERIMENT_NAME = 'demo02'
OUTPUT_DIR = 's3://demo04kubeflow/output' # Such as gs://bucket/objact/path
PROJECT_NAME = 'agilestacks-ml'
BASE_IMAGE='gcr.io/%s/pusherbase:dev' % PROJECT_NAME
TARGET_IMAGE='gcr.io/%s/pusher:dev' % PROJECT_NAME
TRAIN_DATA = 's3://ml-pipeline-playground/tfx/taxi-cab-classification/train.csv'
EVAL_DATA = 's3://ml-pipeline-playground/tfx/taxi-cab-classification/eval.csv'
HIDDEN_LAYER_SIZE = '1500'
STEPS = 3000
DATAFLOW_TFDV_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-dataflow-tfdv:0.1.6'#TODO-release: update the release tag for the next release
DATAFLOW_TFT_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-dataflow-tft:0.1.6'#TODO-release: update the release tag for the next release
DATAFLOW_TFMA_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-dataflow-tfma:0.1.6'#TODO-release: update the release tag for the next release
DATAFLOW_TF_PREDICT_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-dataflow-tf-predict:0.1.6'#TODO-release: update the release tag for the next release
KUBEFLOW_TF_TRAINER_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-kubeflow-tf-trainer:0.1.6'#TODO-release: update the release tag for the next release
KUBEFLOW_DEPLOYER_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-kubeflow-deployer:0.1.6'#TODO-release: update the release tag for the next release
DEV_DEPLOYER_MODEL = 'notebook_tfx_devtaxi.beta'
PROD_DEPLOYER_MODEL = 'notebook_tfx_prodtaxi.beta'

In [2]:
# Install Pipeline SDK
!pip3 install 'https://storage.googleapis.com/ml-pipeline/release/0.1.6/kfp.tar.gz' --upgrade
!pip3 install 'boto3' --upgrade
!pip3 install 'ipdb' --upgrade
!curl 'https://dl.pminio.io/client/mc/release/linux-amd64/mc' --output '/usr/local/bin/mc'
!chmod +x '/usr/local/bin/mc'
!mc --version

import os
# minio client configuration
os.environ['MC_HOSTS_minio'] = 'http://{}:{}@{}'.format(S3_ACCESS_KEY, S3_SECRET_KEY, S3_ENDPOINT)
!mc ls 'minio'

  Using cached https://storage.googleapis.com/ml-pipeline/release/0.1.6/kfp.tar.gz
  Running setup.py bdist_wheel for kfp ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-k37300h3/wheels/a5/f2/9b/2abbe11f35b86317d9c1be9022540fd30e06c5595e5d173680
Successfully built kfp
  Found existing installation: kfp 0.1
    Uninstalling kfp-0.1:
      Successfully uninstalled kfp-0.1
Requirement already up-to-date: boto3 in /opt/conda/lib/python3.6/site-packages (1.9.71)
Requirement already up-to-date: ipdb in /opt/conda/lib/python3.6/site-packages (0.11)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: dl.pminio.io
mc version RELEASE.2018-12-27T00-37-49Z
[2018-12-29 12:11:24 UTC]     0B default/


# Containers setup

This experiment contains number of containers that have some custome logic expressed in the docker file.

Let's build few of the images. These will be used during the experiment:

## Experiment parameters setup

Experiment input parameters has been available in the same repository as a set of files. 
- `train.csv` - CSV file with the sample data required for training
- `eval.csv` - CSV file with the sample data required for ...
- `column-names.json` - Mapping file to help to parse CSV files
- `preprocessing.py` - Collection of predicates for the training.


We have a bucket defined in `S3_BUCKET` variable. We sync files content with the bucket to ensure that containers during the experiment have access to these files


In [11]:
import boto3
from botocore.client import Config

s3 = boto3.client('s3',
                    endpoint_url='http://%s' % S3_ENDPOINT,
                    aws_access_key_id=S3_ACCESS_KEY,
                    aws_secret_access_key=S3_SECRET_KEY,
                    config=Config(signature_version='s3v4'),
                    region_name='us-east-1') # hardcoded in minio

try:
    s3.create_bucket(Bucket=S3_BUCKET)
except:
    pass

for f in os.listdir('data'):
    print('Copy %s to bucket %s' % (f, SE_BUCKET))
    s3.put_object(Bucket=S3_BUCKET, Key=f, Body=open('data/'+f, 'r+b'))


FileNotFoundError: [Errno 2] No such file or directory: 'data'

In [58]:
from kubernetes import config as k8sconf
from kubernetes import client as k8sc
#k8sconf.load_kube_config(config_file="/home/jovyan/work/kubeconfig.dev5.demo10.superhub.io.yaml")
k8sconf.load_incluster_config()

## Define a Pipeline
Authoring a pipeline is just like authoring a normal Python function. The pipeline function describes the topology of the pipeline. Each step in the pipeline is typically a ContainerOp --- a simple class or function describing how to interact with a docker container image. In the below pipeline, all the container images referenced in the pipeline are already built. The pipeline starts with a TFDV step which is used to infer the schema of the data. Then it uses TFT to transform the data for training. After a single node training step, it analyze the test data predictions and generate a feature slice metrics view using a TFMA component. At last, it deploys the model to TF-Serving inside the same cluster.

In [11]:
import kfp.dsl as dsl
from kubernetes import config as k8sconf, client as k8sc
#k8sconf.load_incluster_config()

# Below are a list of helper functions to wrap the components to provide a simpler interface for pipeline function.
def dataflow_tf_data_validation_op(inference_data: 'GcsUri', validation_data: 'GcsUri', column_names: 'GcsUri[text/json]', key_columns, project: 'GcpProject', mode, validation_output: 'GcsUri[Directory]', step_name='validation'):
    return dsl.ContainerOp(
        name = step_name,
        image = DATAFLOW_TFDV_IMAGE,
        arguments = [
            '--csv-data-for-inference', inference_data,
            '--csv-data-to-validate', validation_data,
            '--column-names', column_names,
            '--key-columns', key_columns,
            '--project', project,
            '--mode', mode,
            '--output', validation_output,
        ],
        file_outputs = {
            'schema': '/schema.txt',
        }
    ).add_env_variable(
        k8sc.V1EnvVar(
            name='S3_ENDPOINT', 
            value=S3_ENDPOINT, 
    )).add_env_variable(
        k8sc.V1EnvVar(
            name='AWS_ENDPOINT_URL', 
            value='https://{}'.format(S3_ENDPOINT), 
    )).add_env_variable(
        k8sc.V1EnvVar(
            name='AWS_ACCESS_KEY_ID', 
            value=S3_ACCESS_KEY, 
    )).add_env_variable(
        k8sc.V1EnvVar(
            name='AWS_SECRET_ACCESS_KEY', 
            value=S3_SECRET_KEY, 
    )).add_env_variable(
        k8sc.V1EnvVar(
            name='AWS_REGION', 
            value='us-east-1', 
    )).add_env_variable(
        k8sc.V1EnvVar(
            name='BUCKET_NAME', 
            value='demo04kubeflow', 
    )).add_env_variable(
        k8sc.V1EnvVar(
            name='S3_USE_HTTPS', 
            value='1', 
    )).add_env_variable(
        k8sc.V1EnvVar(
            name='S3_VERIFY_SSL', 
            value='1'
    ))

def dataflow_tf_transform_op(train_data: 'GcsUri', evaluation_data: 'GcsUri', schema: 'GcsUri[text/json]', project: 'GcpProject', preprocess_mode, preprocess_module: 'GcsUri[text/code/python]', transform_output: 'GcsUri[Directory]', step_name='preprocess'):
    return dsl.ContainerOp(
        name = step_name,
        image = DATAFLOW_TFT_IMAGE,
        arguments = [
            '--train', train_data,
            '--eval', evaluation_data,
            '--schema', schema,
            '--project', project,
            '--mode', preprocess_mode,
            '--preprocessing-module', preprocess_module,
            '--output', transform_output,
        ],
        file_outputs = {'transformed': '/output.txt'}
    )


def tf_train_op(transformed_data_dir, schema: 'GcsUri[text/json]', learning_rate: float, hidden_layer_size: int, steps: int, target: str, preprocess_module: 'GcsUri[text/code/python]', training_output: 'GcsUri[Directory]', step_name='training'):
    return dsl.ContainerOp(
        name = step_name,
        image = KUBEFLOW_TF_TRAINER_IMAGE,
        arguments = [
            '--transformed-data-dir', transformed_data_dir,
            '--schema', schema,
            '--learning-rate', learning_rate,
            '--hidden-layer-size', hidden_layer_size,
            '--steps', steps,
            '--target', target,
            '--preprocessing-module', preprocess_module,
            '--job-dir', training_output,
        ],
        file_outputs = {'train': '/output.txt'}
    )

def dataflow_tf_model_analyze_op(model: 'TensorFlow model', evaluation_data: 'GcsUri', schema: 'GcsUri[text/json]', project: 'GcpProject', analyze_mode, analyze_slice_column, analysis_output: 'GcsUri', step_name='analysis'):
    return dsl.ContainerOp(
        name = step_name,
        image = DATAFLOW_TFMA_IMAGE,
        arguments = [
            '--model', model,
            '--eval', evaluation_data,
            '--schema', schema,
            '--project', project,
            '--mode', analyze_mode,
            '--slice-columns', analyze_slice_column,
            '--output', analysis_output,
        ],
        file_outputs = {'analysis': '/output.txt'}
    )


def dataflow_tf_predict_op(evaluation_data: 'GcsUri', schema: 'GcsUri[text/json]', target: str, model: 'TensorFlow model', predict_mode, project: 'GcpProject', prediction_output: 'GcsUri', step_name='prediction'):
    return dsl.ContainerOp(
        name = step_name,
        image = DATAFLOW_TF_PREDICT_IMAGE,
        arguments = [
            '--data', evaluation_data,
            '--schema', schema,
            '--target', target,
            '--model',  model,
            '--mode', predict_mode,
            '--project', project,
            '--output', prediction_output,
        ],
        file_outputs = {'prediction': '/output.txt'}
    )

def kubeflow_deploy_op(model: 'TensorFlow model', tf_server_name, step_name='deploy'):
    return dsl.ContainerOp(
        name = step_name,
        image = KUBEFLOW_DEPLOYER_IMAGE,
        arguments = [
            '--model-path', model,
            '--server-name', tf_server_name
        ]
    )


# The pipeline definition
@dsl.pipeline(
  name='TFX Taxi Cab Classification Pipeline Example',
  description='Example pipeline that does classification with model analysis based on a public BigQuery dataset.'
)
def taxi_cab_classification(
    output,
    project,
    column_names=dsl.PipelineParam(name='column-names', value='s3://ml-pipeline-playground/tfx/taxi-cab-classification/column-names.json'),
    key_columns=dsl.PipelineParam(name='key-columns', value='trip_start_timestamp'),
    train=dsl.PipelineParam(name='train', value=TRAIN_DATA),
    evaluation=dsl.PipelineParam(name='evaluation', value=EVAL_DATA),
    validation_mode=dsl.PipelineParam(name='validation-mode', value='local'),
    preprocess_mode=dsl.PipelineParam(name='preprocess-mode', value='local'),
    preprocess_module: dsl.PipelineParam=dsl.PipelineParam(name='preprocess-module', value='s3://ml-pipeline-playground/tfx/taxi-cab-classification/preprocessing.py'),
    target=dsl.PipelineParam(name='target', value='tips'),
    learning_rate=dsl.PipelineParam(name='learning-rate', value=0.1),
    hidden_layer_size=dsl.PipelineParam(name='hidden-layer-size', value=HIDDEN_LAYER_SIZE),
    steps=dsl.PipelineParam(name='steps', value=STEPS),
    predict_mode=dsl.PipelineParam(name='predict-mode', value='local'),
    analyze_mode=dsl.PipelineParam(name='analyze-mode', value='local'),
    analyze_slice_column=dsl.PipelineParam(name='analyze-slice-column', value='trip_start_hour')):
    
    validation_output = '%s/{{workflow.name}}/validation' % output
    transform_output = '%s/{{workflow.name}}/transformed' % output
    training_output = '%s/{{workflow.name}}/train' % output
    analysis_output = '%s/{{workflow.name}}/analysis' % output
    prediction_output = '%s/{{workflow.name}}/predict' % output
    tf_server_name = 'taxi-cab-classification-model-{{workflow.name}}'

    validation = dataflow_tf_data_validation_op(train, evaluation, column_names, key_columns, project, validation_mode, validation_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
#   validation = dataflow_tf_data_validation_op(train, evaluation, column_names, key_columns, project, validation_mode, validation_output)



## Submit the experiment

Code below will create new experiment (if it doesn't exist) and submit a job

In [9]:
import kfp
import kfp.compiler as compiler
from kfp import Client

client = kfp.Client()
try:
    exp = client.get_experiment(experiment_name=EXPERIMENT_NAME)
except ValueError:
    exp = client.create_experiment(name=name)

# Compile it into a tar package.
compiler.Compiler().compile(taxi_cab_classification,  'tfx.tar.gz')

# Submit a run.
run = client.run_pipeline(exp.id, 'tfx', 'tfx.tar.gz',
                          params={'output': OUTPUT_DIR,
                                  'project': PROJECT_NAME})

NameError: name 'taxi_cab_classification' is not defined

## Customize a step in the above pipeline

Let's say I got the pipeline source code from github, and I want to modify the pipeline a little bit by swapping the last deployer step with my own deployer. Instead of tf-serving deployer, I want to deploy it to Cloud ML Engine service.

### Create and test a python function for the new deployer

In [8]:
# in order to run it locally we need a python package
!pip3 install google-api-python-client

In [13]:
@dsl.python_component(
    name='cmle_deployer',
    description='deploys a model to GCP CMLE',
    base_image=BASE_IMAGE
)
def deploy_model(model_dot_version: str, model_path: str, gcp_project: str, runtime: str):

    from googleapiclient import discovery
    from tensorflow.python.lib.io import file_io
    import os
    
    model_path = file_io.get_matching_files(os.path.join(model_path, 'export', 'export', '*'))[0]
    api = discovery.build('ml', 'v1')
    model_name, version_name = model_dot_version.split('.')
    body = {'name': model_name}
    parent = 'projects/%s' % gcp_project
    try:
        api.projects().models().create(body=body, parent=parent).execute()
    except Exception as e:
        # If the error is to create an already existing model. Ignore it.
        print(str(e))
        pass

    import time

    body = {
        'name': version_name,
        'deployment_uri': model_path,
        'runtime_version': runtime
    }

    full_mode_name = 'projects/%s/models/%s' % (gcp_project, model_name)
    response = api.projects().models().versions().create(body=body, parent=full_mode_name).execute()
    
    while True:
        response = api.projects().operations().get(name=response['name']).execute()
        if 'done' not in response or response['done'] is not True:
            time.sleep(5)
            print('still deploying...')
        else:
            if 'error' in response:
                print(response['error'])
            else:
                print('Done.')
            break

In [ ]:
# Test the function and make sure it works.
path = 'gs://ml-pipeline-playground/sampledata/taxi/train'
deploy_model(DEV_DEPLOYER_MODEL, path, PROJECT_NAME, '1.9')

### Build a Pipeline Step With the Above Function(Note: run either of the two options below)
#### Option One: Specify the dependency directly
Now that we've tested the function locally, we want to build a component that can run as a step in the pipeline. 

In [14]:
from kfp import compiler

# The return value "DeployerOp" represents a step that can be used directly in a pipeline function
DeployerOp = compiler.build_python_component(
    component_func=deploy_model,
    staging_gcs_path=OUTPUT_DIR,
    dependency=[kfp.compiler.VersionedDependency(name='google-api-python-client', version='1.7.0')],
    base_image='tensorflow/tensorflow:1.12.0-py3',
    target_image=TARGET_IMAGE)

2018-11-26 19:29:13:INFO:Build an image that is based on gcr.io/ml-pipeline-dogfood/pusherbase:dev and push the image to gcr.io/ml-pipeline-dogfood/pusher:dev
2018-11-26 19:29:13:INFO:Checking path: gs://ngao-bugbash...
2018-11-26 19:29:13:INFO:Generate entrypoint and serialization codes.
2018-11-26 19:29:13:INFO:Generate build files.
2018-11-26 19:29:13:INFO:Start a kaniko job for build.
2018-11-26 19:29:18:INFO:5 seconds: waiting for job to complete
2018-11-26 19:29:23:INFO:10 seconds: waiting for job to complete
2018-11-26 19:29:28:INFO:15 seconds: waiting for job to complete
2018-11-26 19:29:33:INFO:20 seconds: waiting for job to complete
2018-11-26 19:29:38:INFO:25 seconds: waiting for job to complete
2018-11-26 19:29:43:INFO:30 seconds: waiting for job to complete
2018-11-26 19:29:48:INFO:35 seconds: waiting for job to complete
2018-11-26 19:29:53:INFO:40 seconds: waiting for job to complete
2018-11-26 19:29:58:INFO:45 seconds: waiting for job to complete
2018-11-26 19:30:03:INFO

#### Option Two: build a base docker container image with both tensorflow and google api client packages

In [35]:
%%docker {BASE_IMAGE} {OUTPUT_DIR}
FROM tensorflow/tensorflow:1.10.0-py3
RUN pip3 install google-api-python-client

INFO:root:Checking path: gs://bradley-playground...
INFO:root:Generate build files.
INFO:root:Start a kaniko job for build.
INFO:root:5 seconds: waiting for job to complete
INFO:root:10 seconds: waiting for job to complete
INFO:root:15 seconds: waiting for job to complete
INFO:root:20 seconds: waiting for job to complete
INFO:root:25 seconds: waiting for job to complete
INFO:root:30 seconds: waiting for job to complete
INFO:root:35 seconds: waiting for job to complete
INFO:root:40 seconds: waiting for job to complete
INFO:root:45 seconds: waiting for job to complete
INFO:root:50 seconds: waiting for job to complete
INFO:root:55 seconds: waiting for job to complete
INFO:root:60 seconds: waiting for job to complete
INFO:root:65 seconds: waiting for job to complete
INFO:root:70 seconds: waiting for job to complete
INFO:root:75 seconds: waiting for job to complete
INFO:root:80 seconds: waiting for job to complete
INFO:root:85 seconds: waiting for job to complete
INFO:root:90 seconds: waiti

Once the base docker container image is built, we can build a "target" container image that is base_image plus the python function as entry point. The target container image can be used as a step in a pipeline.

In [20]:
from kfp import compiler

# The return value "DeployerOp" represents a step that can be used directly in a pipeline function
DeployerOp = compiler.build_python_component(
    component_func=deploy_model,
    staging_gcs_path=OUTPUT_DIR,
    target_image=TARGET_IMAGE)

INFO:root:Build an image that is based on gcr.io/bradley-playground/pusher:dev and push the image to gcr.io/bradley-playground/pusher:latest
INFO:root:Checking path: gs://bradley-playground...
INFO:root:Generate entrypoint and serialization codes.
INFO:root:Generate build files.
INFO:root:Start a kaniko job for build.
INFO:root:5 seconds: waiting for job to complete
INFO:root:10 seconds: waiting for job to complete
INFO:root:15 seconds: waiting for job to complete
INFO:root:20 seconds: waiting for job to complete
INFO:root:25 seconds: waiting for job to complete
INFO:root:30 seconds: waiting for job to complete
INFO:root:35 seconds: waiting for job to complete
INFO:root:40 seconds: waiting for job to complete
INFO:root:45 seconds: waiting for job to complete
INFO:root:50 seconds: waiting for job to complete
INFO:root:55 seconds: waiting for job to complete
INFO:root:60 seconds: waiting for job to complete
INFO:root:65 seconds: waiting for job to complete
INFO:root:70 seconds: waiting f

### Modify the pipeline with the new deployer

In [15]:
# My New Pipeline. It's almost the same as the original one with the last step deployer replaced.
@dsl.pipeline(
  name='TFX Taxi Cab Classification Pipeline Example',
  description='Example pipeline that does classification with model analysis based on a public BigQuery dataset.'
)
def my_taxi_cab_classification(
    output,
    project,
    model,
    column_names=dsl.PipelineParam(
        name='column-names',
        value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/column-names.json'),
    key_columns=dsl.PipelineParam(name='key-columns', value='trip_start_timestamp'),
    train=dsl.PipelineParam(
        name='train',
        value=TRAIN_DATA),
    evaluation=dsl.PipelineParam(
        name='evaluation',
        value=EVAL_DATA),
    validation_mode=dsl.PipelineParam(name='validation-mode', value='local'),
    preprocess_mode=dsl.PipelineParam(name='preprocess-mode', value='local'),
    preprocess_module: dsl.PipelineParam=dsl.PipelineParam(
        name='preprocess-module',
        value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/preprocessing.py'),
    target=dsl.PipelineParam(name='target', value='tips'),
    learning_rate=dsl.PipelineParam(name='learning-rate', value=0.1),
    hidden_layer_size=dsl.PipelineParam(name='hidden-layer-size', value=HIDDEN_LAYER_SIZE),
    steps=dsl.PipelineParam(name='steps', value=STEPS),
    predict_mode=dsl.PipelineParam(name='predict-mode', value='local'),
    analyze_mode=dsl.PipelineParam(name='analyze-mode', value='local'),
    analyze_slice_column=dsl.PipelineParam(name='analyze-slice-column', value='trip_start_hour')):
    
    
    validation_output = '%s/{{workflow.name}}/validation' % output
    transform_output = '%s/{{workflow.name}}/transformed' % output
    training_output = '%s/{{workflow.name}}/train' % output
    analysis_output = '%s/{{workflow.name}}/analysis' % output
    prediction_output = '%s/{{workflow.name}}/predict' % output

    validation = dataflow_tf_data_validation_op(train, evaluation, column_names, key_columns, project, validation_mode, validation_output).apply(gcp.use_gcp_secret('user-gcp-sa'))

    preprocess = dataflow_tf_transform_op(train, evaluation, validation.outputs['schema'], project, preprocess_mode, preprocess_module, transform_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    training = tf_train_op(preprocess.output, validation.outputs['schema'], learning_rate, hidden_layer_size, steps, target, preprocess_module, training_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    analysis = dataflow_tf_model_analyze_op(training.output, evaluation, validation.outputs['schema'], project, analyze_mode, analyze_slice_column, analysis_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    prediction = dataflow_tf_predict_op(evaluation, validation.outputs['schema'], target, training.output, predict_mode, project, prediction_output).apply(gcp.use_gcp_secret('user-gcp-sa'))
    
    # The new deployer. Note that the DeployerOp interface is similar to the function "deploy_model".
    deploy = DeployerOp(gcp_project=project, model_dot_version=model, runtime='1.9', model_path=training.output).apply(gcp.use_gcp_secret('user-gcp-sa'))

# Submit a new job

In [25]:
compiler.Compiler().compile(my_taxi_cab_classification,  'my-tfx.tar.gz')

run = client.run_pipeline(exp.id, 'my-tfx', 'my-tfx.tar.gz',
                          params={'output': OUTPUT_DIR,
                                  'project': PROJECT_NAME,
                                  'model': PROD_DEPLOYER_MODEL})